<a href="https://colab.research.google.com/github/Muyikem21/Sunday_INFO5731_Spring2021/blob/main/Copy_of_INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

# New Section

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [ ]:
import pandas as pd
import nltk
from nltk.util import ngrams
import spacy
import en_core_web_sm
from collections import Counter
import numpy as np
from math import log
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import Word
from numpy import linalg as la

datafile = pd.read_csv(r"C:\data.csv")
datafile = datafile[['Reviews', 'RevisedReviews']]
datafile['RevisedReviews'] = datafile['RevisedReviews'].apply(lambda x: nltk.word_tokenize(x))
datafile['N=3 gram'] = datafile['RevisedReviews'].apply(lambda row: list(nltk.ngrams(row, 3)))
datafile['number of N=3 gram'] = datafile['N=3 gram'].str.len()


def prob_func(x):
    probs = []
    twograms = list(nltk.ngrams(x, 2))
    for (w1, w2) in twograms:
        nums = twograms.count((w1, w2))
        for w1 in (w1, w2):
            denoms = x.count(w1)
            probs.append(((w1, w2), nums / denoms))
    return probs

datafile['Two Gram Chance'] = datafile['RevisedReviews'].apply(lambda x: prob_func(x))

datafile.head(5)

nlp = en_core_web_sm.load()

def phrases(vec1):
    vec1 = vec1.replace('', '')
    vec1 = vec1.lower()
    cont1 = []
    cont2 = nlp(vec1)
    parts = list(cont2.noun_parts)
    for part in parts:
        cont1.append(part)
    return cont1

dataframe = pd.DataFrame()
dataframe['Reviews'] = data['Reviews']
dataframe['Phrases'] = dataframe['Reviews'].apply(lambda x: phrases(x))
dataframe['Phrases'] = dataframe['Phrases'].apply(lambda x: [str(item) for item in x])
list = dataframe["Phrases"].tolist()
list2 = [str(item) for sublist in list for item in sublist]

freqlist = Counter(list2)
print('Phrase Frequency :', freqlist.most_common(1)[0])
FreqCount = freqlist.most_common(1)[0][1]

def prob_func2(x):
    prob_phrase = []
    phrase_cont = []
    for Phrases in x:
        if Phrases not in phrase_cont:
            freq = x.count(Phrases)
            rel_prob = round(freq / FreqCount, 5)
            prob_phrase.append((Phrases, rel_prob))
            phrase_cont.append(Phrases)
    return prob_phrase

dataframe['probabilities of phrases'] = dataframe['phrases'].apply(lambda x: prob_func2(x))

dataframe.head(5)





# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [ ]:
rev = pd.read_csv(r"C:\data")
data2 = rev.set_index('ReviewNumber').T.to_dict('Column')

def Conversion(s):
    containerstring = " "
    return (containerstring.join(s))

data2 = {doc_label: Conversion(s) for doc_label, s in data2.items()}

container4 = {doc_label: nltk.word_tokenize(text) for doc_label, text in data2.items()}

numberofoccurances = {doc_label: Counter(tok) for doc_label, tok in container4.items()}

container5 = set()
for counter in numberofoccurances.values():
    container5 |= set(counter.keys())
container5 = sorted(list(container5))

container6 = []
for counter in numberofoccurances.values():
    container6rows = [counter.get(term, 0) for term in container5]
    container6.append(container6rows)
labels = list(numberofoccurances.keys())

container7 = np.mat(container6, dtype=float)
container8 = container7 / np.sum(container7, axis=1)

def function3(t, docs):
    return sum(t in d for d in docs.values())

def function4(t, docs):
    a = function3(t, docs)
    return log(1 + len(docs) / (1+a))

container9 = [function4(t, container4) for t in container5]
container10 = np.mat(np.diag(container9))
container11 = container8 * container10

container12 = np.array(container11)
container13 = pd.DataFrame(data=container12, index=rev['review_id'], columns=container5)
container13.head()

stop = stopwords.words('english')

def function6(query):
    tempcontainer = tempcontainer.replace('[^\w\s]', '')
    tempcontainer = tempcontainer.replace('\d+', '')
    tempcontainer = " ".join(x for x in tempcontainer.split() if x not in stop)
    tempcontainer = " ".join(x.lower() for x in tempcontainer.split())
    tempcontainer = " ".join([PorterStemmer().stem(word) for word in tempcontainer.split()])
    tempcontainer = " ".join([Word(word).lemmatize() for word in tempcontainer.split()])
    return tempcontainer

def function14(query):
    tempcontainer2 = {'query': function6(query)}

    tempcontainer3 = {doc_label: nltk.word_tokenize(text) for doc_label, text in tempcontainer2.items()}
    tempcontainer4 = {doc_label: Counter(tok) for doc_label, tok in tempcontainer3.items()}

    container20 = []
    for counter in tempcontainer4.values():
        container21 = [counter.get(term, 0) for term in container5]
        container20.append(container21)
    container22 = list(tempcontainer4.keys())

    container23 = np.mat(container20, dtype=float)
    container24 = container23 / np.sum(container23, axis=1)

    container25 = [function4(t, tempcontainer3) for t in container5]
    container26 = np.mat(np.diag(container25))
    container27 = container24 * container26

    container28 = np.array(container27)
    return container28

def cossimfunc(input1, input2):
    input2 = np.array(input2)
    input1 = function14(input1)
    if la.norm(input1) != 0 and la.norm(input2) != 0:
        cossimis = np.inner(input1, input2) / la.norm(input1) * la.norm(input2)
    else:
        cossimis = 0
    return float(cossimis)


stringcontainer = "the movie is extraordinary"

cossimcontainer = []
for i in range(len(container13)):
    cossimcontainer2 = cossimfunc(stringcontainer, container13.iloc[i])
    cossimcontainer.append(cossimcontainer2)

cossimresult = pd.DataFrame({'The Review': labels, 'Cosine Similarity Result': cossimcontainer})
cossimresult.head()








# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [ ]:


Sunday_INFO5731_Spring2021/THEASSIGNMENT.csv
